# Import Libraries

In [ ]:
!pip install gensim

In [ ]:
!pip install pyldavis

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade gensim

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using 

In [ ]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import multiprocessing
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Load spaCy for lemmatization
nlp = spacy.load('en_core_web_sm')

# Load Data

In [ ]:
pd.set_option('max_colwidth', None)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/vjavaly/Baruch-CIS-9665/refs/heads/main/data/NIPS_papers_1500.csv')

In [ ]:
df.shape

(1500, 6)

In [ ]:
df.head(3)

year  \
0  1997   
1  2007   
2  2017   

                                                                                                  title  \
0  Independent Component Analysis for Identification of Artifacts in Magnetoencephalographic Recordings   
1                       Near-Maximum Entropy Models for Binary Neural Representations of Natural Images   
2                     Nearest-Neighbor Sample Compression: Efficiency, Consistency, Infinite Dimensions   

  event_type  \
0        NaN   
1        NaN   
2     Poster   

                                                                                                        pdf_name  \
0  1466-independent-component-analysis-for-identification-of-artifacts-in-magnetoencephalographic-recordings.pdf   
1                       3336-near-maximum-entropy-models-for-binary-neural-representations-of-natural-images.pdf   
2                        6755-nearest-neighbor-sample-compression-efficiency-consistency-infinite-dimensions.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   abstract  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Abstract Missing   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Abstract Missing   
2  We examine the Bayes-consistency of a recently proposed 1-nearest-neighbor-based multiclass learning algorithm. This algorithm is derived from sample compression bound

In [ ]:
df.isna().sum()

,0
year,0
title,0
event_type,1000
pdf_name,0
abstract,0
paper_text,0


## Create New Dataframe

In [ ]:
df2 = df['title'] + ' ' + df['paper_text']

In [ ]:
df2.head(3)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

# Preprocess Text

In [ ]:
def preprocess(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()

        # Replace newline and multiple whitespaces with a single space
        text = re.sub(r'\s+', ' ', text)

        # Remove digits
        text = re.sub(r'\d+', '', text)

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        # Keep tokens with more than 2 characters
        tokens = [word for word in tokens if len(word) > 2]

        # Lemmatize using spaCy
        lemmatized_tokens = [token.lemma_ for token in nlp(" ".join(tokens)).doc]

        return lemmatized_tokens

    else:
        return []  # Return an empty list if the text is not a valid string

In [ ]:
%%time

processed_text = df2.apply(preprocess)

CPU times: user 9min 22s, sys: 41.2 s, total: 10min 3s
Wall time: 10min 20s


In [ ]:
processed_text.head(3)

,0
0,"[independent, component, analysis, identification, artifact, magnetoencephalographic, recording, independent, component, analysis, identification, artifact, magnetoencephalographic, recording, ricardo, vigario, veikko, ousmiiki, matti, hiimiiliiinen, riitta, hari, erkki, oja, lab, computer, info, science, helsinki, university, technology, box, fin, hut, finland, ricardovigario, erkkiojahutfi, brain, research, unit, low, temperature, lab, helsinki, university, technology, box, fin, hut, finland, veikko, msh, harineurohutfi, abstract, study, application, independent, component, analysis, ica, approach, identification, possible, removal, artifact, magnetoencephalographic, meg, recording, statistical, technique, separate, component, accord, kurtosis, amplitude, distribution, time, thus, distinguish, strictly, periodical, signal, regularly, irregularly, occur, signal, many, artifact, belong, last, category, order, assess, effectiveness, method, control, artifact, produce, include, saccadic, ...]"
1,"[nearmaximum, entropy, model, binary, neural, representation, natural, image, nearmaximum, entropy, model, binary, neural, representation, natural, image, matthias, bethge, philipp, berens, max, planck, institute, biological, cybernetics, spemannstrasse, tubingen, germany, mbethgeberenstuebingenmpgde, abstract, maximum, entropy, analysis, binary, variable, provide, elegant, way, study, role, pairwise, correlation, neural, population, unfortunately, approach, suffer, poor, scalability, high, dimension, sensory, coding, however, highdimensional, datum, ubiquitous, introduce, new, approach, use, nearmaximum, entropy, model, make, type, analysis, feasible, highdimensional, datathe, model, parameter, derive, closed, form, sampling, easy, therefore, nearmaxent, approach, serve, tool, testing, prediction, pairwise, maximum, entropy, model, lowdimensional, marginal, also, high, dimensional, measurement, thousand, unit, demonstrate, usefulness, study, natural, ...]"
2,"[nearestneighbor, sample, compression, efficiency, consistency, infinite, dimension, nearestneighbor, sample, compression, efficiency, consistency, infinite, dimensions, aryeh, kontorovich, department, computer, science, bengurion, university, negev, karyehcsbguacil, sivan, sabato, department, computer, science, bengurion, university, negev, sabatosbguacil, roi, weiss, department, computer, science, applied, mathematics, weizmann, institute, science, roiwweizmannacil, abstract, examine, bayesconsistency, recently, propose, nearestneighborbase, multiclass, learning, algorithm, algorithm, derive, sample, compression, bound, enjoy, statistical, advantage, tight, fully, empirical, generalization, bound, well, algorithmic, advantage, fast, runtime, memory, saving, prove, algorithm, strongly, bayesconsistent, metric, space, finite, double, dimension, first, consistency, result, efficient, nearestneighbor, sample, compression, scheme, rather, surprisingly, discover, algorithm, continue, bayesconsistent, even, certain, infinitedimensional, set, basic, ...]"


# Generate Gensim Dictionary Object

In [ ]:
%%time

# Map each word in to its unique integer id
dictionary = Dictionary(processed_text)

CPU times: user 2.85 s, sys: 35.3 ms, total: 2.89 s
Wall time: 2.89 s


In [ ]:
print("{} words in the Dictionary object.".format(len(dictionary)))

98496 words in the Dictionary object.


In [ ]:
# Remove very rare and very common words
#  Filter out tokens that appear in:
#   < 5 documents (remove words that appear in fewer than 5 documents)
#   > 40% documents (remove words that appear in more than 40% of documents)
dictionary.filter_extremes(no_below=5, no_above=0.4)
print("{} words remaining in the Dictionary object.".format(len(dictionary)))

14214 words remaining in the Dictionary object.


In [ ]:
# Convert documents into the bag-of-words format: list of (token_id, token_count) 2-tuples
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_text]

# Train LDA Model (BOW Corpus)

In [ ]:
%%time

# Train model with Bag of Words corpus
lda_model_bow = LdaMulticore(corpus=bow_corpus, id2word=dictionary, num_topics=10,
                             chunksize=10, passes=10, gamma_threshold=0.001,
                             per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

CPU times: user 23.1 s, sys: 1.5 s, total: 24.6 s
Wall time: 39.6 s


# Evaluate LDA Model (BOW Corpus)

In [ ]:
%%time

# Compute Coherence Score: c_v
coherence_model_lda = CoherenceModel(model=lda_model_bow, texts=processed_text,
                                     dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print('LDA BOW Coherence Score (c_v): ', coherence_score)

LDA BOW Coherence Score (c_v):  0.565995761119147
CPU times: user 1min 5s, sys: 368 ms, total: 1min 5s
Wall time: 1min 6s


In [ ]:
%%time

# Compute Coherence Score: u_mass
coherence_model_lda = CoherenceModel(model=lda_model_bow, texts=processed_text,
                                     dictionary=dictionary, coherence='u_mass')
coherence_score = coherence_model_lda.get_coherence()
print('LDA BOW Coherence Score (u_mass): ', coherence_score)

LDA BOW Coherence Score (u_mass):  -1.451321881876052
CPU times: user 1.65 s, sys: 90.8 ms, total: 1.74 s
Wall time: 1.74 s


In [ ]:
# Display topics identified by LDA model
all_topics = lda_model_bow.print_topics()
for topic in all_topics:
    print(topic)

(0, '0.034*"kernel" + 0.028*"label" + 0.017*"classifier" + 0.011*"risk" + 0.011*"hypothesis" + 0.010*"regression" + 0.008*"generalization" + 0.008*"svm" + 0.008*"margin" + 0.007*"unlabeled"')
(1, '0.024*"object" + 0.010*"human" + 0.008*"detection" + 0.007*"face" + 0.007*"visual" + 0.007*"pixel" + 0.007*"pose" + 0.006*"shape" + 0.006*"similarity" + 0.006*"vision"')
(2, '0.013*"bayesian" + 0.012*"topic" + 0.011*"likelihood" + 0.011*"posterior" + 0.011*"inference" + 0.010*"mixture" + 0.010*"density" + 0.008*"covariance" + 0.007*"variational" + 0.007*"latent"')
(3, '0.024*"neuron" + 0.018*"cell" + 0.016*"spike" + 0.014*"response" + 0.013*"stimulus" + 0.010*"activity" + 0.009*"subject" + 0.009*"filter" + 0.008*"field" + 0.008*"frequency"')
(4, '0.013*"convex" + 0.012*"sparse" + 0.011*"norm" + 0.009*"rank" + 0.007*"objective" + 0.006*"min" + 0.006*"lasso" + 0.005*"regression" + 0.005*"projection" + 0.005*"sparsity"')
(5, '0.043*"layer" + 0.035*"deep" + 0.021*"arxiv" + 0.016*"batch" + 0.014*"

In [ ]:
# Display topics using 'show_topics' which provides more options
#   Returns the top 5 topics with the top 5 words per topic
top_topics = lda_model_bow.show_topics(num_topics=5, num_words=5, formatted=False)
for topic in top_topics:
    print(topic)

(7, [('policy', 0.026056727), ('action', 0.023655426), ('regret', 0.01807593), ('online', 0.015356237), ('reward', 0.015254039)])
(5, [('layer', 0.04332829), ('deep', 0.035299692), ('arxiv', 0.021327922), ('batch', 0.015569905), ('convolutional', 0.0142770335)])
(2, [('bayesian', 0.012753932), ('topic', 0.011678577), ('likelihood', 0.011361216), ('posterior', 0.011206238), ('inference', 0.010802712)])
(6, [('graph', 0.04257518), ('cluster', 0.03853856), ('node', 0.029979745), ('tree', 0.024852268), ('edge', 0.016040184)])
(8, [('memory', 0.015948422), ('hide', 0.010934705), ('dynamic', 0.008899505), ('net', 0.008893962), ('circuit', 0.007909868)])


# LDA Model (BOW Corpus) Visualization

In [ ]:
# Create Topic Distance Visualization
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.080539 -0.021906       1        1  16.373917
4      0.110350 -0.125521       2        1  15.275599
1     -0.040252  0.105068       3        1  12.894464
3     -0.009363  0.269203       4        1  10.421259
0      0.119324 -0.078182       5        1  10.324308
7      0.120301 -0.078713       6        1   9.144024
8     -0.026961  0.112059       7        1   8.802756
6      0.137264  0.000568       8        1   7.988753
5     -0.242689 -0.217349       9        1   4.989102
9     -0.248514  0.034772      10        1   3.785817, topic_info=                Term         Freq        Total Category  logprob  loglift
2723           graph  5452.000000  5452.000000  Default  30.0000  30.0000
1009          kernel  6609.000000  6609.000000  Default  29.0000  29.0000
2429         cluster  5341.000000  5341.000000  Default  28.0000  28.0000
1262            word  4019.000000  4019.000000  Default  27.0000  27.0000
1681           layer  4987.000000  4987.000000  Default  26.0000  26.0000
...              ...          ...          ...      ...      ...      ...
86    dimensionality   299.117322  1019.192508  Topic10  -5.2892   2.0480
294       similarity   324.338812  1950.237822  Topic10  -5.2082   1.4800
2268           digit   269.562229   593.993046  Topic10  -5.3932   2.4838
676        reduction   289.366997  1125.012775  Topic10  -5.3223   1.9161
299           source   298.600593  2417.557885  Topic10  -5.2909   1.1825

[656 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1273     10  0.990563       acl
3704      7  0.016337  acoustic
3704     10  0.980202  acoustic
2948      3  0.003713    action
2948      4  0.028564    action
...     ...       ...       ...
9492      6  0.855082    worker
8555      7  0.994356       wta
2222      1  0.995891       xed
8997      7  0.988351       xtl
2224      9  0.993921    yoshua

[1545 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 4, 1, 8, 9, 7, 6, 10])

## Run TF-IDF on Bag of Words

In [ ]:
# Create tf-idf model object on ‘bow_corpus’
tfidf_model = TfidfModel(bow_corpus)

In [ ]:
# Apply transformation to entire corpus
tfidf_corpus = tfidf_model[bow_corpus]

# Train LDA Model (TF-IDF Corpus)

In [ ]:
# Train model with TFIDF corpus
lda_model_tfidf = LdaMulticore(corpus=tfidf_corpus, id2word=dictionary, num_topics=10,
                               chunksize=10, passes=10, gamma_threshold=0.001,
                               per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

# Evaluate LDA Model (TF-IDF Corpus)

In [ ]:
%%time

# Compute Coherence Score: c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=processed_text,
                                     dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print('LDA TF-IDF Coherence Score (c_v): ', coherence_score)

LDA TF-IDF Coherence Score (c_v):  0.4623956468622895
CPU times: user 43.5 s, sys: 194 ms, total: 43.7 s
Wall time: 44.1 s


In [ ]:
%%time

# Compute Coherence Score: u_mass
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=processed_text,
                                     dictionary=dictionary, coherence='u_mass')
coherence_score = coherence_model_lda.get_coherence()
print('LDA TF-IDF Coherence Score (u_mass): ', coherence_score)

LDA TF-IDF Coherence Score (u_mass):  -10.795315632718133
CPU times: user 1.78 s, sys: 31.6 ms, total: 1.81 s
Wall time: 1.83 s


# KeyBERT Label Extraction (BOW Corpus)

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Initialize KeyBERT
kw_model = KeyBERT()

# Extracting Labels
topic_labels = []
for topic_num, topic_str in all_topics:
    # Extract words
    words = re.findall(r'"(.*?)"', topic_str)
    joined_words = " ".join(words)
    # Extract keyword with KeyBERT
    label = kw_model.extract_keywords(joined_words, keyphrase_ngram_range=(1, 2), top_n=1)

    # Decide what to store
    if label:
        keyword, score = label[0]
        topic_labels.append((topic_num, keyword, round(score * 100, 2)))  # percent format
    else:
        topic_labels.append((topic_num, "not found", 0.0))

print('Topic Labels:\n')
for topic_num, keyword, percent in topic_labels:
    print(f"Topic {topic_num}: {keyword} ({percent}%)")

Topic Labels:

Topic 0: margin unlabeled (60.44%)
Topic 1: detection face (57.34%)
Topic 2: topic likelihood (70.87%)
Topic 3: spike response (63.29%)
Topic 4: convex sparse (70.56%)
Topic 5: deep arxiv (56.84%)
Topic 6: belief vertex (61.35%)
Topic 7: arm strategy (57.55%)
Topic 8: architecture trajectory (48.98%)
Topic 9: decoder context (58.88%)


# Perform BERTopic
### Compare BERTopic results to LDA Model (BOW Corpus) results

In [ ]:
! pip install bertopic

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic()

/usr/local/lib/python3.11/dist-packages/geopandas/_compat.py:7: DeprecationWarning: The 'shapely.geos' module is deprecated, and will be removed in a future version. All attributes of 'shapely.geos' are available directly from the top-level 'shapely' namespace (since shapely 2.0.0).
  import shapely.geos


In [ ]:
# Train the BERTopic model
topics, probs = topic_model.fit_transform(processed_text.apply(lambda x: ' '.join(x)))

In [ ]:
# Reduce topics
bertopic_top_N = topic_model.reduce_topics(processed_text.apply(lambda x: ' '.join(x)), nr_topics=6)
bertopic_top_N.get_topics()

{-1: [('use', 0.0289810236266652),
  ('model', 0.027545234542551082),
  ('algorithm', 0.0237778795143963),
  ('datum', 0.02321750213501457),
  ('set', 0.02312467646143305),
  ('function', 0.019195629982956444),
  ('learn', 0.018672192243484955),
  ('problem', 0.018359942301477427),
  ('distribution', 0.017414292683146378),
  ('result', 0.016928864884554272)],
 0: [('use', 0.029237780522055543),
  ('algorithm', 0.025822167366303935),
  ('function', 0.023374721691471888),
  ('model', 0.02319547786811835),
  ('set', 0.021707384926221117),
  ('learn', 0.020936078592496462),
  ('network', 0.020311158500494458),
  ('problem', 0.017678243798680555),
  ('method', 0.017461537799667214),
  ('result', 0.016914761948985294)],
 1: [('neuron', 0.04291756943407936),
  ('model', 0.03848672360383348),
  ('spike', 0.03157382621573059),
  ('cell', 0.024182945950691613),
  ('neural', 0.022228392117500873),
  ('stimulus', 0.022074571456401897),
  ('input', 0.021637242669441496),
  ('use', 0.021137764301853

In [ ]:
# Visualize word bar chart for reduced topics
bertopic_top_N.visualize_barchart(top_n_topics=6, n_words=5)